# Sphere Evolution Analysis 

Script to analyse data of sphere evolution with scale factor, using density growth, normalised density growth compared to approximated growth modes, residuals for this comparison, slice plots and gradient plots. The scipt creates a number of functions to complete this analysis and then at the end loops over different sphere radii and calls the functions to produce plots. 


In [ ]:
#Import packages
import matplotlib.pyplot as plt
import numpy as np 
from scipy.optimize import curve_fit as scurv
from scipy.stats import norm 
from scipy import interpolate as interp
import matplotlib.colors as colors

In [ ]:
#Define functions to use later in plotting/curve fitting
def resids(true, approx):
    y = (true-approx)/approx
    return y 

def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - (x - mean)**2 / (2*standard_deviation ** 2))

## Load in data for given radius
This cell manpiulates data so it correctly sorted for plotting as well as in array object type, and can be passed to other functions. 

In [ ]:
def load_data(rad):
    
    z = []
    q = open(f'Text_data/z_{rad}.txt', 'r')
    lines = q.readlines()
    for i in lines: 
        z.append(i.split(  )[0])

    q.close()
    z_arr = np.array(z).astype(float)
    sort = np.argsort(z_arr)
    z_arr = z_arr[sort]
    a = 1/(1+z_arr)

    deltas = []
    k = open(f'Text_Data/delta_{rad}.txt', 'r')
    lines = k.readlines()
    for i in lines:
        deltas.append(i.split(  ))

    k.close()
    delta_arr = np.array(deltas).astype(float)
    delta_arr = delta_arr[sort]
    
    lahav_arr = []
    q = open(f'Text_Data/Extrapolation Data/lahav_{rad}Mpc.txt', 'r')
    lines = q.readlines()
    for i in lines: 
        lahav_arr.append(i.split(  )[0])
        
    q.close()
    lahav_arr = np.array(lahav_arr).astype(float)
    lahav_arr = np.sort(lahav_arr)
    lahav_arr = np.flip(lahav_arr)
    lahav_norm = lahav_arr[0]
    lahav_normal = lahav_arr/lahav_norm
    
    carroll_arr = []
    q = open(f'Text_Data/Extrapolation Data/carroll_{rad}Mpc.txt', 'r')
    lines = q.readlines()
    for i in lines: 
        carroll_arr.append(i.split(  )[0])
        
    q.close()
    carroll_arr = np.array(carroll_arr).astype(float)
    carroll_arr = np.sort(carroll_arr)
    carroll_arr = np.flip(carroll_arr)
    carroll_norm = carroll_arr[0]
    carroll_normal = carroll_arr/carroll_norm
    
    ana_arr = []
    q = open(f'Text_Data/Extrapolation Data/analytic_{rad}Mpc.txt', 'r')
    lines = q.readlines()
    for i in lines: 
        ana_arr.append(i.split(  )[0])
        
    q.close()

    ana_arr = np.array(ana_arr).astype(float)
    ana_arr = np.sort(ana_arr)
    ana_arr = np.flip(ana_arr)
    ana_norm = ana_arr[0] 
    ana_normal = ana_arr/ana_norm
    
    return delta_arr, carroll_normal, lahav_normal, ana_normal, a

## Plotting density evolution of every sphere over redshift values

In [ ]:
def evolution_plot(delta_arr, rad):
    plt.plot(a, delta_arr, alpha = 0.2)
    plt.title("Sphere Overdensity as a function of redshift")
    plt.xlabel("Scale factor a")
    plt.ylabel("Overdensity $\delta$")
    plt.savefig(f"Misc/Spread_{rad}Mpc.jpeg", dpi = 200) 
    plt.show()
    plt.clf()

## Normalised growth of sphere density compared to approximations

In [ ]:
def normal_plot(caroll_normal, delta_arr, lahav_normal, ana_normal, rad, a):
    delta_norm= delta_arr/delta_arr[0]
    delta_norm = np.array(delta_norm).T
    for k in range(1000):
        plt.plot(a, delta_norm[k], alpha = 0.05)
    delta_norm = delta_norm.astype(float)

    
    plt.plot(a, carroll_normal, color = 'r', label = 'Carroll', linestyle = 'dashed')
    plt.plot(a, lahav_normal, color = 'b', label =' Lahav', linestyle = 'dashed')
    plt.plot(a, ana_normal, color = 'g', label = 'Analytic Growth', linestyle='dashed')
    plt.ylim(0,1)
    plt.xlabel('Scale Factor a')
    plt.ylabel('Normalised Density $\delta$/$\delta_{0}$')
    plt.title('Density Evolution Comparison to Approximations')
    plt.legend()
    plt.savefig(f"Approx_Plots/Sphere_comparison_{rad}Mpc.jpeg", dpi = 200)
    plt.clf()
    
    return delta_norm

## Residuals of normalised growth plots against approximations

In [ ]:
def residuals(caroll_normal, delta_norm, lahav_normal, rad, ana_normal):
    ten = []
    above = []
    chisq_sphere = np.array([])
    for i in range(1000):
        res = resids(delta_norm[i], carroll_normal)
        if (res< 0.1).all() and (res> -0.1).all():
            plt.plot(a,res, alpha = 0.05, color = 'r')
            ten.append(res)
        else:
            plt.plot(a, res, alpha= 0.05, color = 'k')
            
        chisq = np.sum(res**2)
        chisq_sphere = np.append(chisq_sphere, chisq)
    chisq_sphere[939], chisq_sphere[92], chisq_sphere[810], chisq_sphere[456] = 0, 0, 0, 0  
    chi_total_c = np.sum(chisq_sphere)
        

    print(f'There are {len(ten)} spheres within 10% of Carroll for {rad}Mpc spheres.')

    plt.ylim(-1.5,1.5)
    plt.hlines(0, 0, 1, colors = 'w', linestyle ='dashed')
    plt.xlabel('Scale Factor a')
    plt.ylabel('Residuals')
    #plt.title('Residuals plot for Carroll approximation')
    plt.savefig(f'Residuals/Carroll_resids_{rad}Mpc.jpeg', dpi = 200)
    #plt.show()
    plt.clf()

    ten = []
    above= []
    chisq_sphere = np.array([])
    for i in range(1000):
        res = resids(delta_norm[i], lahav_normal)

        if (res< 0.1).all() and (res> -0.1).all():
            plt.plot(a,res, alpha = 0.05, color = 'r')
            ten.append(res)  
        else:
            plt.plot(a, res, alpha= 0.05, color = 'k')
            
        chisq = np.sum(res**2)
        chisq_sphere = np.append(chisq_sphere, chisq)
       

    chisq_sphere[939], chisq_sphere[92], chisq_sphere[810], chisq_sphere[456] = 0, 0, 0,0  
    chi_total_l = np.sum(chisq_sphere)
            
    print(f'There are {len(ten)} spheres within 10% of Lahav for {rad}Mpc spheres.')


    plt.ylim(-1.5,1.5)
    plt.hlines(0, 0, 1, colors = 'w', linestyle ='dashed')
    plt.xlabel('Scale Factor a')
    plt.ylabel('Residuals')
    plt.title('Residuals plot for Lahav approximation')
    plt.savefig(f'Residuals/Lahav_resids_{rad}Mpc.jpeg', dpi = 200)
    #plt.show()
    plt.clf()
    
    return chi_total_c, chi_total_l

## Plotting the number of over- and under-dense spheres

In [ ]:
def number(delta_arr, rad, a):
    
    upper = []
    lower = []
    for i in range(len(a)):
        mask = delta_arr[i] > 0
        above = len(delta_arr[i][mask])
        below = 1000 - above
        upper.append(above)
        lower.append(below)

    plt.clf()
    #plt.title(f'Overdense and Underdense Spheres for r = {rad}Mpc')
    plt.plot(a, (upper), color = 'r', label = 'Overdense')
    plt.plot(a, (lower), color = 'k', label = 'Underdense')
    plt.xlabel('Scale Factor a')
    plt.ylabel('Number of spheres')
    plt.legend()
    plt.savefig(f"Misc/OverUnder_{rad}Mpc.jpeg", dpi =200)
    plt.show()
    plt.clf()

## Histograms of density over all spheres for each scale factor with gaussian overplotting 

In [ ]:
def slice_plots(delta_norm, carroll_normal, lahav_normal, rad, a):
    std_arr = []
    x = np.linspace(0,1,2000)
    delta_norm = delta_norm.T
    z_arr = 1/a -1
    
    b = [8]
    for i in b:
        n_bin = 70
        z = np.round(z_arr[i], decimals = 2)
        bin_heights, bin_borders = np.histogram(delta_norm[i], bins = 70, range = (0,1))
        bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 

        hist =plt.hist(delta_norm[i], bins = n_bin, range = (0,1), label = 'Overdensities', color = 'g' )
        #plt.title(f'Slice Plot for {rad}Mpc Spheres at a = {np.round(a[i], decimals = 2)}')
        plt.xlabel('$\delta$/$\delta_{0}$')
        plt.ylabel('Frequency')
        ymin, ymax = plt.ylim()
        plt.vlines(carroll_normal[i], ymin, ymax, colors = 'r', linestyle = 'dashed', label = f'Carroll z = {z} ')
        popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [np.mean(delta_norm[i]), 1 , 0.05], maxfev = 100000)
        plt.plot(x, gaussian(x, *popt), label = f'Gaussian Fit', color = 'b')
        plt.legend()
        plt.savefig(f'Slice_Plots/slice_plot_{rad}Mpc_{i}.jpeg', dpi = 200  )
        plt.show()
        

        std = norm.fit(delta_norm[i])[1]
        std = popt[2]
        std_arr.append(std)

        plt.clf()
    """
    std_arr = np.array(std_arr)


    plt.plot(a, np.absolute(std_arr), color = 'b')
    plt.xlabel('Scale Factor a')
    plt.ylabel('Standard Deviation $\sigma$')
    plt.title('Spread of Data for Normalised Densities')
    plt.savefig(f'Variance_Plots/Normalised_Spread_{rad}Mpc.jpeg', dpi = 200)
    plt.show()
    plt.clf()
    """
    return std_arr

## Gradient of sphere growth for comparison with approximation gradient

In [ ]:
def gradient(delta_norm, carroll_normal, lahav_normal, rad):
    

    lahav_grad = np.gradient(lahav_normal)
    carroll_grad = np.gradient(carroll_normal)
    for i in range((delta_norm.shape[0])):
        gradient = np.gradient(delta_norm[i]) 
        plt.plot(a, gradient, alpha = 0.05, color = 'b')

    plt.plot(a, lahav_grad, color = 'orange', label = 'Lahav')
    plt.plot(a, carroll_grad, color = 'r', label = 'Carroll')
    plt.ylabel('Gradient of Evolution Lines')
    plt.xlabel('Scale Factor a')
    plt.ylim(-0.5,0.5)
    plt.legend()
    plt.savefig(f'Misc/Gradients_{rad}Mpc.jpeg', dpi = 200)

    plt.show()

## Create heatmap for better representation of normalised density. 

In [ ]:
def heatmap(delta_arr, a_plot, carroll_norm, rad):
    
    fig, ax = plt.subplots(figsize = (12,6), ncols = 3, nrows = 1, sharey = True)
    for j in range(len(a_plot)):
        a = a_plot[j]
        sort = np.argsort(a)
        a = a[sort]
        carroll_normal = carroll_norm[j]
        carroll_normal = carroll_normal[sort]


        zero_hist = np.zeros((50,50))
        n_hist = 1000
        delta_norm = delta_arr[j]
        for i in range(n_hist):

            delta_norm[i] = delta_norm[i][sort]
            Int = interp.UnivariateSpline(a, (delta_norm[i]), s = 0.1, k=2)
            k = np.linspace(0,1,1000)
            hist, xedges, yedges = np.histogram2d((k), (Int(k)), bins=50)
            zero_hist += hist 

        
        norm = colors.Normalize(vmin=0, vmax=2500)

        m = (carroll_normal[1]-carroll_normal[0])/(a[1]-a[0])
        y = m*(a - a[1]) + carroll_normal[1]
        ax[j].plot(a, (carroll_normal), color = 'white', linestyle='--', label = 'Carroll')
        ax[j].plot(a, y, color = 'r', linestyle='--', label = 'Analytic')
        im = ax[j].imshow(zero_hist.T, extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]], origin='lower', 
                   cmap='Greens', aspect = 'auto', alpha = 1, norm = norm) 
        ax[j].set_title(f'{rad[j]} Mpc/h')
        ax[j].set_ylim(0,1)
    
    fig.text(0.5, 0.02, 'Scale Factor a', ha='center', fontsize=14)
    fig.text(0.04, 0.5, '$\delta$/$\delta_{0}$', va='center', rotation='vertical', fontsize=14)
    fig.subplots_adjust(wspace=0.15, hspace=0.15)
    plt.savefig(f'Subplots\Heatmap_Subplots_{rad}Mpc', dpi = 200) 
    plt.show()
    

## Call functions in a loop over sphere radii to produce analysis plots

In [ ]:
rads = [30, 50,70]

chi_c, chi_l = [], [] 
deltas = []
a_arr = []
carroll = []
std = []
for rad in rads:
    delta_arr, carroll_normal, lahav_normal, ana_normal, a  = load_data(rad)
    evolution_plot(delta_arr, rad)
    delta_norm = normal_plot(carroll_normal, delta_arr, lahav_normal, ana_normal, rad,a)
    deltas.append(delta_norm)
    a_arr.append(a)
    carroll.append(carroll_normal)
    chisq_c, chisq_l = residuals(carroll_normal, delta_norm, lahav_normal, rad, ana_normal)
    chi_c.append(chisq_c)
    chi_l.append(chisq_l)
    number(delta_arr, rad, a)
    std_arr = slice_plots(delta_norm, carroll_normal, lahav_normal, rad, a)
    std.append(std_arr)
    gradient(delta_norm, carroll_normal, lahav_normal, rad)
    
heatmap(deltas, a_arr, carroll, rads)


In [ ]:
for i in range(len(std)):
    plt.plot(a, np.abs(std[i]), label = f'{rads[i]} Mpc/h')
plt.legend()
plt.xlabel('Scale Factor a')
plt.ylabel('Standard Deviation $\sigma$')
plt.savefig('Misc/slice_sigmas.jpeg', dpi = 200)